<p style="font-family:verdana;font-size:200%;text-align:center;">Linear Regression</p>

### 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import numpy as np
import pandas as pd

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# xlsx 파일을 읽고, 데이터프레임 df를 생성합니다.
df = pd.read_excel(io = 'Used_Cars_Price_Prep.xlsx')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
dm = pd.get_dummies(data = df['FuelType'], drop_first = True)

In [ ]:
# df의 오른쪽에 더미변수 dm을 추가합니다.
df = pd.concat(objs = [df, dm], axis = 1)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df에서 FuelType을 삭제합니다.
df = df.drop(labels = ['FuelType'], axis = 1)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 목표변수 열이름을 설정합니다.
yvar = 'Price'

In [ ]:
# 입력변수 행렬 X와 목표변수 벡터 y를 생성합니다.
X = df.drop(labels = [yvar], axis = 1)
y = df[yvar]

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [ ]:
# 훈련셋의 목표변수 평균을 확인합니다.
y_tr.mean().round(2)

In [ ]:
# 시험셋의 목표변수 평균을 확인합니다.
y_te.mean().round(2)

### 선형 회귀모형 적합

In [ ]:
# 관련 라이브러리를 호출합니다.
import statsmodels.api as sm

In [ ]:
# 훈련셋 입력변수에 y절편의 역할을 수행할 상수 1을 추가합니다.
X_tr = sm.add_constant(data = X_tr)

In [ ]:
# 선형 회귀모형을 반환하는 함수를 정의합니다.
def ols(y, X):
    model = sm.OLS(endog = y, exog = X)
    return model.fit()

### 선형 회귀모형 적합 결과 확인

In [ ]:
# 훈련셋으로 선형 회귀모형 적합 결과를 확인합니다.
fit1 = ols(y = y_tr, X = X_tr)
fit1.summary()

### 다중공선성 확인

In [ ]:
# 관련 모듈을 호출합니다.
import HelloDataScience as hds

In [ ]:
# 훈련셋의 입력변수별 분산팽창지수를 출력합니다.
hds.vif(X = X_tr)

In [ ]:
# 분산팽창지수가 5 이상인 입력변수를 훈련셋에서 삭제합니다.
X_tr = X_tr.drop(labels = ['Petrol'], axis = 1)

In [ ]:
# 훈련셋의 입력변수별 분산팽창지수를 다시 출력합니다.
hds.vif(X = X_tr)

### 후진소거법으로 변수 선택

In [ ]:
# 변경된 훈련셋으로 선형 회귀모형 적합 결과를 확인합니다.
fit2 = ols(y = y_tr, X = X_tr)
fit2.summary()

In [ ]:
# 회귀계수의 유의성 검정을 통과하지 못한 입력변수를 모두 제거합니다.
X_tr = X_tr.drop(labels = ['MetColor', 'Doors'], axis = 1)

In [ ]:
# 변경된 훈련셋으로 선형 회귀모형 적합 결과를 확인합니다.
fit3 = ols(y = y_tr, X = X_tr)
fit3.summary()

### 표준화 회귀계수 확인

In [ ]:
# 선형 회귀모형의 회귀계수만 출력합니다.
fit3.params

In [ ]:
# 입력변수의 표준편차를 목표변수의 표준편차로 나눈 값을 출력합니다.
X_tr.std() / y_tr.std()

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = fit3.params * (X_tr.std() / y_tr.std())

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순으로 정렬하여 출력합니다.
beta_z.abs().sort_values()

### 목표변수의 추정값 생성

In [ ]:
# 훈련셋의 추정값을 생성합니다.
y_tr_pred = fit3.predict(exog = X_tr)

In [ ]:
# 훈련셋의 추정값 일부를 출력합니다.
y_tr_pred[0:5]

In [ ]:
# 훈련셋의 실제값과 비교합니다.
y_tr[0:5]

In [ ]:
# 훈련셋에서 삭제했던 입력변수를 시험셋에서도 삭제합니다.
X_te = X_te.drop(labels = ['MetColor', 'Doors', 'Petrol'], axis = 1)

In [ ]:
# 시험셋에 상수항을 추가합니다.
X_te = sm.add_constant(data = X_te)

In [ ]:
# 시험셋의 추정값을 생성합니다.
y_te_pred = fit3.predict(exog = X_te)

In [ ]:
# 시험셋의 추정값 일부를 출력합니다.
y_te_pred[0:5]

In [ ]:
# 시험셋의 실제값과 비교합니다.
y_te[0:5]

### 회귀모형 성능 평가

In [ ]:
# 훈련셋의 실제값과 추정값으로 회귀모형의 성능을 평가합니다.
hds.regmetrics(y_true = y_tr, y_pred = y_tr_pred)

In [ ]:
# 시험셋의 실제값과 추정값으로 회귀모형의 성능을 평가합니다.
hds.regmetrics(y_true = y_te, y_pred = y_te_pred)

<p style="font-family:verdana;font-size:200%;text-align:center;">End of Document</p>